In [5]:
import json
from collections import Counter
from pathlib import Path
import pandas as pd

dataset_root = "D:\David\Škola\RP\dataset"
train_filename = "train_no_dup.json"
valid_filename = "valid_no_dup.json"
test_filename = "test_no_dup.json"

counts = []

def process_dataset(dataset_filename):
    outfits_disposed = 0
    total_items = 0
    total_outfits = 0
    categories = []
    with open(Path(dataset_root, dataset_filename)) as json_file:
        raw_json = json.load(json_file)
        print("Loaded " + str(len(raw_json)) + " items", flush=True)
        examples = []
        for outfit in raw_json:
            images = []
            counts.append(len(outfit["items"]))
            for item in outfit["items"]:
                total_items = total_items + 1
                categories.append(item["categoryid"])
                images.append(1)
            if len(images) < 3:
                outfits_disposed = outfits_disposed + 1
                continue
            
            total_outfits = total_outfits + 1


    print("Total " + str(total_items) + " products")
    print("Total " + str(total_outfits) + " outfits")
    return examples

process_dataset(train_filename)
process_dataset(valid_filename)
process_dataset(test_filename)

counter = Counter(counts)
series = pd.Series(counts)
print(series.describe())
print(counter)

Loaded 17316 items
Total 114806 products
Total 17316 outfits
Loaded 1497 items
Total 9070 products
Total 1497 outfits
Loaded 3076 items
Total 18604 products
Total 3076 outfits
count    21889.000000
mean         6.509206
std          1.401401
min          4.000000
25%          5.000000
50%          7.000000
75%          8.000000
max          8.000000
dtype: float64
Counter({8: 7867, 6: 4028, 7: 3903, 5: 3691, 4: 2400})


In [ ]:
def search_unique_items():
    with open(Path(dataset_root, dataset_filename)) as json_file:
        raw_json = json.load(json_file)
        print("Loaded " + str(len(raw_json)) + " outfits", flush=True)
        unique_ids = {}
        duplicates = {}
        total_items = 0
        outfits_without_unique = 0
        for outfit in raw_json:
            for item in outfit["items"]:
                total_items = total_items + 1
                _, id = item['image'].split('id=')
                if id not in unique_ids:
                    unique_ids[id] = True
                else:
                    duplicates[id] = True
                    
        for outfit in raw_json:
            for item in outfit["items"]:
                _, id = item['image'].split('id=')
                if id in duplicates:
                    outfits_without_unique += 1
                    
        print("Total " + str(total_items))
        print("Unique " + str(len(unique_ids)))
        print("outfits_without_unique " + str(outfits_without_unique))
                    
search_unique_items()